## Libraries

In [34]:
import os
import time
import tracemalloc as tm

import polars as pl
import pandas as pd
import duckdb

import plotly.express as px
import matplotlib.pyplot as plt

## Raw Files [Path]

In [35]:
raw_main_data = r'Data_extracted\student_habits_performance.csv'

## Test Functions

In [84]:
def read_files(path: str, reader: str) -> dict:
    finalTime = None
    df = None
    startTime = time.time()
    reader = reader.lower()

    if reader  == 'duckdb':

        global con_duck
        con_duck = duckdb.connect(database = ":memory:")
        con_duck.execute(f'CREATE TABLE datatable AS FROM read_csv_auto("{raw_main_data}")')
        df = con_duck.execute("SELECT * FROM datatable").fetchdf()

    elif reader  == 'pandas':
        df = pd.read_csv(path)
        
    elif reader  == 'polars':
        df = pl.read_csv(path, ignore_errors=True)

    endTime = time.time()
    finalTime = endTime - startTime
    assert len(df) > 0
    
    return {
            'reader':reader,
            'reading_time': finalTime,
            'data': df
            }

def data_populating(data, reader: str, register_multipliyer: int) -> dict:
    finalTime = None
    df_concated = None
    startTime = time.time()
    reader = reader.lower()

    if reader == 'duckdb':
        con_duck.register("my_data", data)

        union_query = " UNION ALL ".join(["SELECT * FROM my_data"] * register_multipliyer)

        df_concated = con_duck.execute(union_query).fetchdf()

    elif reader == 'pandas':
        df_concated = pd.concat([data]* register_multipliyer, ignore_index=True)

    elif reader == 'polars':

        df_concated = pl.concat([data] * register_multipliyer)

    endTime = time.time()
    
    finalTime = endTime - startTime

    assert len(df_concated) == register_multipliyer*len(data)
    
    return {
            'reader':reader,
            'reading_time': finalTime,
            'data' : df_concated
            }

def data_grouping(data, reader: str, grouping_columns: list, agg_columns: list) -> dict:
    finalTime = None
    df_concated = None
    agg_columns = [f'SUM({col})' for col in agg_columns] if reader.lower() == 'duckdb' else agg_columns
    startTime = time.time()
    reader = reader.lower()

    if reader == 'duckdb':            
        df_grouped = con_duck.execute(f"SELECT {','.join(grouping_columns)},{','.join(agg_columns)} FROM datatable GROUP BY {','.join(grouping_columns)}").fetchdf()

    elif reader == 'pandas':
        df_grouped = data.groupby(by=grouping_columns)[agg_columns].sum()

    elif reader == 'polars':

        agg_exprs = [pl.col(col).sum() for col in agg_columns]
        
        df_grouped = data.group_by(grouping_columns).agg(agg_exprs)

    endTime = time.time()

    
    finalTime = endTime - startTime

    return {
            'reader':reader,
            'reading_time': finalTime,
            'data' : df_grouped
            }

def data_filtering(data, reader: str, filter_column: list) -> dict:
    finalTime = None
    df_filtered = None
    startTime = time.time()
    reader = reader.lower()

    if reader == 'duckdb':  

        df_filtered = con_duck.execute(f"SELECT * FROM datatable WHERE {filter_column} > (SELECT AVG({filter_column}) FROM datatable)").fetchdf()

    elif reader == 'pandas':

        df_filtered = data[data[filter_column] > data[filter_column].mean()]

    elif reader == 'polars':
        
        df_filtered = data.filter(data[filter_column] > data[filter_column].mean())

    endTime = time.time()

    
    finalTime = endTime - startTime

    return {
            'reader':reader,
            'reading_time': finalTime,
            'data' : df_filtered
            }

def column_sorting(data, reader: str, sort_columns: list):
    finalTime = None
    df_sorted = None
    startTime = time.time()
    reader = reader.lower()

    if reader == 'duckdb':            
        df_sorted = con_duck.execute(f"SELECT * FROM datatable ORDER BY {','.join(sort_columns)}").fetchdf()

    elif reader == 'pandas':
        df_sorted = data.sort_values(sort_columns)

    elif reader == 'polars':
        df_sorted = data.sort(sort_columns)
        

    endTime = time.time()

    
    finalTime = endTime - startTime

    return {
            'reader':reader,
            'reading_time': finalTime,
            'data' : df_sorted
            }

def format_to_parquet(data, reader: str, saving_path: str):
    finalTime = None
    startTime = time.time()
    reader = reader.lower()

    if reader == 'duckdb':

        con_duck.execute(f"""
                            COPY (
                                SELECT * FROM datatable
                            )
                            TO '{saving_path}_duckdb.parquet' (FORMAT 'parquet');
                        """)

    elif reader == 'pandas':

        data.to_parquet(f'{saving_path}_polars.parquet')

    elif reader == 'polars':

        data.write_parquet(f'{saving_path}_polars.parquet')

    endTime = time.time()

    finalTime = endTime - startTime

    return {
            'reader':reader,
            'reading_time': finalTime
            }

## Reading Speed

In [37]:
readers = ['polars','pandas','duckDB']
reading_results = {}

for reader in readers:
    reading_results[reader] = read_files(path = raw_main_data, reader = reader)

graph_results = {}
for reader in readers:
    graph_results[reader] = reading_results[reader]["reading_time"]
    print(f'{reader}: {reading_results[reader]["reading_time"]}') 

polars: 0.008883476257324219
pandas: 0.0037963390350341797
duckDB: 0.03666496276855469


In [38]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))


## Register Increment Speed

In [ ]:
increment_results = {}
list_memory_usage = {}

for reader in readers:
    tm.start()
    increment_results[reader] = data_populating(data = reading_results[reader]["data"], reader = reader, register_multipliyer = 600)
    current_memory, peak_memory = tm.get_traced_memory()
    tm.stop()
    list_memory_usage[reader] = {'current_memory': current_memory, 'peak_memory': peak_memory}


graph_results = {}
for reader in readers:
    graph_results[reader] = increment_results[reader]["reading_time"]
    print(f'{reader}: {increment_results[reader]["reading_time"]}') 
    print(f'{reader}: {len(increment_results[reader]["data"])}') 


polars: 0.023172616958618164
polars: 600000
pandas: 0.21195149421691895
pandas: 600000
duckDB: 9.075915575027466
duckDB: 600000


In [40]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))

## Data Grouping Speed

In [41]:
grouping_columns = ['age','diet_quality']
agg_columns = ['study_hours_per_day','social_media_hours']

grouping_results = {}

for reader in readers:
    grouping_results[reader] = data_grouping(data = increment_results[reader]["data"], reader = reader, grouping_columns = grouping_columns, agg_columns = agg_columns)

graph_results = {}
for reader in readers:
    graph_results[reader] = grouping_results[reader]["reading_time"]
    print(f'{reader}: {grouping_results[reader]["reading_time"]}') 
    print(f'{reader}: {len(grouping_results[reader]["data"])}') 

polars: 0.16227006912231445
polars: 24
pandas: 0.03031468391418457
pandas: 24
duckDB: 0.009162187576293945
duckDB: 24


In [42]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))

## Column Sorting Speed

In [43]:
sort_columns = ['age','study_hours_per_day']

column_sorting_results = {}

for reader in readers:
    column_sorting_results[reader] = column_sorting(data = increment_results[reader]["data"], reader = reader, sort_columns = sort_columns)

graph_results = {}
for reader in readers:
    graph_results[reader] = column_sorting_results[reader]["reading_time"]
    print(f'{reader}: {column_sorting_results[reader]["reading_time"]}') 
    print(f'{reader}: {len(column_sorting_results[reader]["data"])}') 

polars: 0.08291196823120117
polars: 600000
pandas: 0.10739636421203613
pandas: 600000
duckDB: 0.003770589828491211
duckDB: 1000


In [44]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))

## Data Filtering Speed

For the filtering column selection it needs to full the following requirements:
- Be a *numeric* column
- Uses a single column to make a single condition filter

And the internal parameters for it to filter are:
- It will only check for values greater than the mean of the column

In [45]:
column_to_filter = 'age'

filter_columns_results = {}

for reader in readers:
    filter_columns_results[reader] = data_filtering(data = increment_results[reader]["data"], reader = reader, filter_column = column_to_filter)

graph_results = {}
for reader in readers:
    graph_results[reader] = filter_columns_results[reader]["reading_time"]
    print(f'{reader}: {filter_columns_results[reader]["reading_time"]}') 
    print(f'{reader}: {len(filter_columns_results[reader]["data"])}') 

polars: 0.05608987808227539
polars: 294600
pandas: 0.02198648452758789
pandas: 294600
duckDB: 0.0
duckDB: 491


In [49]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))

## Memory Usage

In [46]:
data_for_plot = []
for lib, mems in list_memory_usage.items():
    for mem_type, value in mems.items():
        data_for_plot.append({
            'library': lib,
            'memory_type': 'Current' if mem_type == 'current_memory' else 'Peak',
            'memory': value
        })

In [47]:
fig = px.bar(
    data_for_plot,
    x='library',
    y='memory',
    color='memory_type',
    barmode='group',
    labels={'memory': 'Memory Usage (bytes)', 'library': 'Library', 'memory_type': 'Memory Type'},
    title='Memory Usage by Library'
)
fig.update_yaxes(type='log')

fig.show()

## Data Offload Speed

In [83]:
path = './Data_Offload/data'
data_offload_results = {}

for reader in readers:
    data_offload_results[reader] = format_to_parquet(data = increment_results[reader]["data"], reader = reader, saving_path = path)

graph_results = {}
for reader in readers:
    graph_results[reader] = data_offload_results[reader]["reading_time"]
    print(f'{reader}: {data_offload_results[reader]["reading_time"]}') 

polars
duckdb
polars: 0.10904526710510254
pandas: 4.636104106903076
duckDB: 0.0051517486572265625


In [85]:
px.bar(x=list(graph_results.keys()),y=list(graph_results.values()))